In [1]:
import pandas as pd
import numpy as np
from simpletransformers.classification import MultiLabelClassificationArgs
from multi_label_classification_model_wrapper import MultiLabelClassificationModelWrapper
import sklearn
import sklearn.metrics
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import re
import os
from mbti_util import MbtiUtil
from denoicer import Denoicer

In [2]:
mbti_util = MbtiUtil()
denoice = Denoicer()

In [3]:
# 感情コーパスを読み込み訓練データとテストデータに分割する
def read_dataset_to_frame():
    all_data = []
    edf = pd.read_csv('./database/emotion/emotion_vector_corpus_ones.tsv', sep='\t')
    emost = 'Joy Sadness Anticipation Surprise Anger Fear Disgust Trust'
    edf_shuf = sklearn.utils.shuffle(edf, random_state=1)
    for s, e in edf_shuf[['Sentence', emost]].values:
        mc = re.match(r'.*nan.*', emost)
        if mc != None:
            continue
        all_data.append([denoice.normalize_text(s.strip()), np.array(e.split(' '), dtype=np.float32)])
    train_data, eval_data = train_test_split(all_data, random_state=1) # 訓練データ（75%）テストデータ（25%）に分割
    train_df = pd.DataFrame(train_data, columns=['text', 'labels'])
    print("read train data.")
    eval_df = pd.DataFrame(eval_data, columns=['text', 'labels'])
    print("read eval data.")
    return train_df, eval_df

In [4]:
# 感情推定モデルの学習
def train(model_type, model_name, train_df, eval_df):
    model_args = MultiLabelClassificationArgs(
        num_train_epochs=5, 
        train_batch_size=64, 
        eval_batch_size=16,
        use_early_stopping=True
    )
    model = MultiLabelClassificationModelWrapper(
        model_type=model_type,
        model_name=model_name,
        num_labels=8,
        args=model_args,
        use_cuda=True
    )
    model_name = model_name.replace('/', '-')
    if not os.path.exists(f'./model/{model_name}'):
        os.mkdir(f'./model/{model_name}')

    # Train the model
    model.train_model(train_df, output_dir=f'./model/{model_name}')
    # Evaluate the model
    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    return result, model_outputs, wrong_predictions

In [5]:
# 感情推定モデルの推定
def estimate(model_type, model_name, to_predict):
    model = MultiLabelClassificationModelWrapper(
        model_type = model_type,
        model_name = model_name,
        num_labels=8,
        use_cuda = True
    )
    predictions, raw_outputs = model.predict(to_predict=to_predict)
    outputs = []
    for ro in raw_outputs:
        outputs.append(np.round(ro, decimals=5))
    return predictions, outputs

In [6]:
train_df, eval_df = read_dataset_to_frame()

read train data.
read eval data.


In [ ]:
'''
以下のいずれかのコメントを外して学習するモデルを選択する
'''
model_type, model_name = ('roberta_waseda_ja', 'nlp-waseda/roberta-base-japanese')
# model_type, model_name = ('twhinbert', 'Twitter/twhin-bert-base')
# model_type, model_name = ('bert', 'cl-tohoku/bert-base-japanese-whole-word-masking')
# model_type, model_name = ('xlnet', 'hajime9652/xlnet-japanese')
# model_type, model_name = ('xlmroberta', 'xlm-roberta-base')

result, model_outputs, wrong_predictions = train(
    model_type=model_type, model_name=model_name, train_df=train_df, eval_df=eval_df)

In [100]:
model = MultiLabelClassificationModelWrapper(
        model_type ='twhinbert',
        model_name ='./model/Twitter-twhin-bert-base/checkpoint-7555-epoch-5/',
        num_labels=8,
        use_cuda = True
    )

In [101]:
y_true = []
for i in eval_df['labels'].values:
    v = []
    for j in i:
        v.append(int(j))
    y_true.append(v)
y_true = np.array(y_true)
y_true

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [102]:
predictions, raw_outputs = model.predict(eval_df['text'].to_list())

100%|██████████| 2015/2015 [00:35<00:00, 56.34it/s]


In [123]:
disgust = 6
trust = 7
indexs = []
with open('./tmp.txt', 'w', encoding='utf8') as f:
    f.write('text\ty_true\tpred\n')

for i, (t, p) in enumerate(zip(y_true, predictions)):
    if t[disgust] != p[disgust]:
        with open('./tmp.txt', 'a', encoding='utf8') as f:
            text = eval_df['text'].values[i]
            f.write(f'{text}\t{t}\t{p}\n')

In [95]:
target_names = ['Joy', 'Sadness', 'Anticipation', 'Surprise', 'Anger', 'Fear', 'Disgust', 'Trust']
print(sklearn.metrics.classification_report(y_true, predictions, target_names=target_names))

              precision    recall  f1-score   support

         Joy       0.88      0.85      0.87     12193
     Sadness       0.64      0.55      0.59      4813
Anticipation       0.80      0.74      0.77      9715
    Surprise       0.70      0.56      0.62      3353
       Anger       0.69      0.64      0.66      3907
        Fear       0.60      0.38      0.46      2795
     Disgust       0.79      0.55      0.65      2596
       Trust       0.75      0.56      0.64      4119

   micro avg       0.78      0.67      0.72     43491
   macro avg       0.73      0.60      0.66     43491
weighted avg       0.77      0.67      0.72     43491
 samples avg       0.75      0.72      0.72     43491



/mnt/data1/home/kishima/Covid-19/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/data1/home/kishima/Covid-19/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
def read_use_users()-> list:
    use_users: list = []
    with open('./database/use_users.txt', 'r', encoding='utf-8') as f:
        use_users = list(map(lambda x: int(x.strip()), f.readlines()))
    return use_users

In [5]:
author_ids = []
m_types = []
tweets = []
tweet_ids = []
use_users: list = read_use_users()

mbti_users = pd.read_csv('./database/mbti_users.tsv' ,sep='\t')
for id, m_type in zip(mbti_users['author_id'], mbti_users['mbti_type']):
    if mbti_util.ToEngType(m_type) != "" and id in use_users:
        with open(f'./database/tweets/before/{mbti_util.ToEngType(m_type)}/{id}.txt', 'r', encoding='utf-8') as f:
            lines = f.readlines()[1:]
            for l in range(len(lines)):
                author_ids.append(id)
                m_types.append(m_type)
            befores = list(map(lambda x: x.split('\t')[1].replace('\n', '').strip(), lines))
            tweet_ids = tweet_ids + list(map(lambda x: x.split('\t')[0].strip(), lines))
        tweets = tweets + befores
        with open(f'./database/tweets/after/{mbti_util.ToEngType(m_type)}/{id}.txt', 'r', encoding='utf-8') as f:
            lines = f.readlines()[1:]
            for l in range(len(lines)):
                author_ids.append(id)
                m_types.append(m_type)
            afters = list(map(lambda x: x.split('\t')[1].replace('\n', '').strip(), lines))
            tweet_ids = tweet_ids + list(map(lambda x: x.split('\t')[0].strip(), lines))
        tweets = tweets + afters
df = pd.DataFrame([author_ids, m_types, tweet_ids, tweets]).T
df.columns = ['author_id', 'm_type', 'tweet_id', 'tweet_text']
df

,author_id,m_type,tweet_id,tweet_text
0,940954542576885760,仲介者,1197852103021981696,@yryr_8945 担当制なんだ！笑この量でこのクオリティは本当すごい🎅お疲れ様です☺️
1,940954542576885760,仲介者,1197318394946543617,@yryr_8945 えっ！自作！すごい！
2,940954542576885760,仲介者,1196803484332388352,@stella_kakerun 明日休みだから行ってみる😭ありがとう！
3,940954542576885760,仲介者,1196785937482575874,ファミマ行く暇なかったよー😭
4,940954542576885760,仲介者,1196389448344207370,シュヴァルツヴェルダーキルシュトルテかな？ https://t.co/ggxZQ2v6hL
...,...,...,...,...
442217,523937950,起業家,1343553479860604928,まぐろ おかね がくり https://t.co/fGT1b3TwiX
442218,523937950,起業家,1343182580942655488,弾き納めでした！2020年あっという間だったなあ
442219,523937950,起業家,1342051488558764032,今日はレッスン納め👏「明日家族のクリスマスパーティーで弾くからクリスマスの曲教えて！」って今...
442220,523937950,起業家,1340666066175856646,チラシに載せるタイプの地図の作り方を習得した🙌最近macbookちゃんがスペック面と容量面共...


In [19]:
def read_tweets_with_before_after(isReply=False):
    path = './database/tweets_with_before_or_after.tsv'
    df = pd.read_csv(path, sep='\t')
    if isReply:
        df = df[df['tweet_text'].str.match('@(\w+) ')]
    else:
        df = df[df['tweet_text'].str.match('(?!@(\w+) )')]
    return df

df = read_tweets_with_before_after(isReply=True)
to_predict = df['tweet_text'].values
to_predict = list(map(lambda x: denoice.normalize_text(x), to_predict))
to_predict

['担当制なんだ!笑この量でこのクオリティは本当すごいお疲れ様です',
 'えっ!自作!すごい!',
 '明日休みだから行ってみるありがとう!',
 '国外逃亡',
 'ずっと送れてなかったごめんね!sweets dis parfyってところだよ!ヘザーの隣!',
 '私はsuper beaverの嬉しい涙聞いてたよ',
 '買うんですねルークさん。',
 '大変だよルークさん',
 'うん。スッキリしたから大丈夫。ありがとね',
 'むーたさんありがとうございます色々重なってしまって...幼馴染の胸を借りてきます',
 '金木犀の香水なら新宿で売ってるよ!',
 '毎度ご苦労様です',
 'おめでとう',
 '私もずっと行こうって言ってて行けてなくて早く行きたいです笑でも受験の方が大事だから大人しく待機してます',
 'りんごちゃん頑張ってね!終わったらみんなでカラオケ行こ!',
 '0年前くらいに行きました!笑ありがとうございます!ただいまー!',
 'こんばんは。以前はお世話になりました。当日は私は忙しくていけないので、よろしければお手伝い致します',
 'ゴジラの映画館の近くなんだけど名前忘れた今度新宿行ったら一緒に食べに行こ!',
 '新宿に美味しい魚介系ラーメンがあるよ!',
 'よしのぶさんおめでとうございます',
 'めっちゃ光栄ありがとう',
 'ストーリーはネタバレになっちゃうので次にあったときにお話しますね!全体的なな演出が素晴らしかったです!!音の強弱とか、光の演出とか、いい意味でゾワッてくる感じでした!',
 '本当それあぁ!そっか!コスの写真があったね!私もまた会いたい!カラオケ行こうね!笑',
 '本当に!やったー私も会いたい!',
 'しなのん今日スケステ?!私も今日のソワレ行くよー',
 'お譲り頂けるとの事大変嬉しく思います!フォローありがとうございます。こちらからもフォロー失礼致します。',
 'ぽちなん様、はじめまして。検索より失礼致します。こちらのチケットをお譲り頂きたくお声がけ致しました。当方チケットが0枚も当選せず、切実に探しておりました。お席はどこでも構いません。ご検討よろしくお願い致します。',
 'さーしゃんお誕生日おめでとう!',
 '美味しいですよね',
 'ありがとうございますおお!楽しんできてくださいね是非

In [20]:
# 推定する分のリスト作成
# df = pd.read_csv('./database/reply_origin_tweets.tsv', sep='\t', lineterminator='\n')
# to_predict = list(df['tweet_text'].apply(lambda x: denoice.normalize_text(x)))

# 正規化後に空になったツイートのインデックスのリストを作成
empty_tweet_index = []
for i, pred in enumerate(to_predict):
    if pred == '':
        empty_tweet_index.append(i)

In [21]:
def map_emo_result(predictions) -> list:
    emostr = 'Joy Sadness Anticipation Surprise Anger Fear Disgust Trust'
    emos = emostr.split(' ')
    emo_results = []
    for p in predictions:
        rese = []
        for ei, x in enumerate(p):
            if x > 0.0:
                rese.append(emos[ei])
        es = 'NULL'
        if len(rese) > 0:
            es = '|'.join(rese)
        emo_results.append(es)
    return emo_results

In [22]:
def outputs_emotion_estimate(path, emo_results, raw_outputs, tweet_ids, empty_tweet_index):
    with open(path, 'w', encoding='utf-8') as f:
        f.write('tweet_id\temo\tvec\n')
        for i, (emo, vec, id) in enumerate(zip(emo_results, raw_outputs, tweet_ids)):
            if i not in empty_tweet_index:
                vec = f'{vec[0]},{vec[1]},{vec[2]},{vec[3]},{vec[4]},{vec[5]},{vec[6]},{vec[7]}'
                f.write(f'{id}\t{emo}\t{vec}\n')
    print(f'complete {path}')

In [23]:
'''
model_types = [
    'bert', 
    'xlnet',
    'robertawasedaja',
    'robertaja',
    'twhinbert',
    'xlmroberta'
    ]
model_names = [
    './model/cl-tohoku-bert-base-japanese-whole-word-masking/checkpoint-7555-epoch-5/',
    './model/hajime9652-xlnet-japanese/checkpoint-7555-epoch-5/',
    './model/nlp-waseda-roberta-base-japanese/checkpoint-7555-epoch-5/',
    './model/rinna-japanese-roberta-base/checkpoint-7555-epoch-5/',
    './model/Twitter-twhin-bert-base/checkpoint-7555-epoch-5/',
    './model/xlm-roberta-base/checkpoint-7555-epoch-5/'
    ]
'''
model_types = [
    'twhinbert',
    'xlmroberta'
    ]
model_names = [
    './model/Twitter-twhin-bert-base/checkpoint-7555-epoch-5/',
    './model/xlm-roberta-base/checkpoint-7555-epoch-5/'
    ]
for t, n in zip(model_types, model_names):
    predictions, raw_outputs = estimate(t, n, to_predict)
    emo_results = map_emo_result(predictions)
    model_name = n.split('/')[2]
    outputs_emotion_estimate(f'./database/emotion/estimate_result/{model_name}.tsv', emo_results, raw_outputs, df['tweet_id'], empty_tweet_index)

100%|██████████| 8298/8298 [02:29<00:00, 55.59it/s]


complete ./database/emotion/estimate_result/Twitter-twhin-bert-base.tsv


100%|██████████| 8298/8298 [02:20<00:00, 58.87it/s]


complete ./database/emotion/estimate_result/xlm-roberta-base.tsv
